# Импорты

In [ ]:
# !pip install psycopg2 -q


In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy
import time
import socket
import sys

from creds import db_config

# Подключение к базе и получение данных

In [ ]:
# Импортируем конфигурацию из отдельного файла
try:
    display("✅ Конфигурация загружена из creds.py")
except ImportError:
    display("❌ Файл creds.py не найден")
    # Можно создать fallback конфиг или запросить данные у пользователя
    db_config = {
        'user': input("Введите имя пользователя: "),
        'pwd': input("Введите пароль: "),
        'host': input("Введите хост: "),
        'port': input("Введите порт: "),
        'db': input("Введите название БД: ")
    }

In [ ]:
def check_host_availability():
    """Проверка доступности хоста"""
    try:
        socket.create_connection((db_config['host'], db_config['port']), timeout=10)
        display("✅ Хост доступен")
        return True
    except socket.error as e:
        display(f"❌ Хост недоступен: {e}")
        sys.exit("ОСТАНОВКА: Не удалось подключиться к базе данных. Дальнейшее выполнение невозможно.")

check_host_availability()

In [ ]:
# Создание строки подключения
connection_string = f"postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"
# engine = create_engine(connection_string)
engine = sqlalchemy.create_engine(connection_string)

# Получение всех таблиц
def load_all_tables(engine):
    """Загружает все таблицы из базы данных"""
    
    # Список таблиц для загрузки
    tables = {
        'collisions': 'SELECT * FROM collisions',
        'parties': 'SELECT * FROM parties', 
        'vehicles': 'SELECT * FROM vehicles'
    }
    
    dataframes = {}
    
    for table_name, query in tables.items():
        try:
            print(f"Загружаем таблицу {table_name}...")
            dataframes[table_name] = pd.read_sql(query, engine)
            print(f"✅ {table_name}: {dataframes[table_name].shape[0]} строк, {dataframes[table_name].shape[1]} столбцов")
        except Exception as e:
            print(f"❌ Ошибка при загрузке {table_name}: {e}")
    
    return dataframes

# Загрузка всех данных
print("Начинаем загрузку данных из базы...")
df_dict = load_all_tables(engine)

# Сохранение в отдельные переменные для удобства
collisions = df_dict['collisions']
parties = df_dict['parties']
vehicles = df_dict['vehicles']

print("\n" + "="*50)
print("СВОДКА ПО ДАННЫМ:")
print("="*50)
print(f"collisions: {collisions.shape}")
print(f"parties:    {parties.shape}")
print(f"vehicles:   {vehicles.shape}")

# Просмотр первых строк каждой таблицы
print("\nПервые 3 строки каждой таблицы:")
print("\ncollisions:")
display(collisions.head(3))

print("\nparties:")
display(parties.head(3))

print("\nvehicles:")
display(vehicles.head(3))

# Базовая информация о таблицах
print("\nИнформация о таблицах:")
print("\ncollisions info:")
collisions.info()

print("\nparties info:")
parties.info()

print("\nvehicles info:")
vehicles.info()